In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

def check(n_point, df):
    N = n_point
    df = df
    check_constraint1 = []
    check_constraint2 = []
    check_constraint3 = []

    check_constraint1_check = False
    check_constraint2_check = False
    check_constraint3_check = False

    for k in range(len(df)):
        #c[0][0]
        for i in range(N):
            check_tempc1_1 = dimod.quicksum(df['sample'][k][f'c[{i}][{j}]'] for j in range(N) if i!=j)
            if check_tempc1_1 != 1:
                check_constraint1_check = True
        for i in range(N):
            check_tempc2_1 = dimod.quicksum(df['sample'][k][f'c[{j}][{i}]'] for j in range(N) if i!=j)
            if check_tempc2_1 != 1:
                check_constraint2_check = True
        if check_constraint1_check == True:
            check_constraint1.append(k)
            check_constraint1_check = False
        if check_constraint2_check == True:
            check_constraint2.append(k)
            check_constraint2_check = False

    num_index = []
    for i in range(len(df)):
        for c1 in check_constraint1:
            num_index.append(c1)
            #num_index = np.delete(num_index, c1)
        for c2 in check_constraint2:
            num_index.append(c2)
        for c3 in check_constraint3:
            num_index.append(c3)

    if num_index == []:
        num_index = np.unique(num_index)
        feasible_index = np.arange(0,len(df),1)
    else:
        num_index = np.unique(num_index)
        feasible_index = np.arange(0,len(df),1)
        feasible_index = np.delete(feasible_index, num_index)

    min_energy = 1e9
    min_index = 0
    for i in feasible_index:
        if (min_energy > df['energy'][i]) and (df['energy'][i]>=0):
            min_energy = df['energy'][i]
            min_index = i
    if len(feasible_index) == 0:
        done = False#True # infeasible
    else:
        done = True #False # feasible
    #print(done)
    #print(df['energy'][min_index])
    energycal = 0
    for i in range(N):
        for j in range(N):
            for k in range(N):
                    energycal += distances[i][j]*df['sample'][min_index][f'c[{i}][{k}]']*df['sample'][min_index][f'c[{j}][{(k+1)%N}]']

    success_probability = len(feasible_index)/len(df)
    return df['sample'][min_index], df['energy'][min_index], done, energycal, success_probability#df['energy'][min_index] #min_index, energycal


In [21]:
# Get the distance matrix
from scipy.spatial import distance_matrix

def TSP_instance(n_customer: int, seed: int = 0):

    n_point = n_customer + 1
    assert n_point >= 1

    # 재현성
    rng = np.random.RandomState(seed)

    # 무작위 좌표 생성
    df = pd.DataFrame({
        'x': rng.randint(0, 100, n_point),
        'y': rng.randint(0, 100, n_point),
    })
    # depot (index 0) 고정
    df.loc[0, 'x'] = 0
    df.loc[0, 'y'] = 0

    D = distance_matrix(df[['x', 'y']].values, df[['x', 'y']].values)
    return np.asarray(D, dtype=float)


In [22]:
from dimod import ConstrainedQuadraticModel, CQM, Binary, quicksum, BinaryQuadraticModel, ExactCQMSolver
from dwave.samplers import SteepestDescentSampler, SimulatedAnnealingSampler
import dimod
import neal
import numpy as np
#import greedy

In [23]:
from pyqubo import Array, Placeholder, Constraint

In [24]:
# ---------- QUBO build ----------
def tsp_pyqubo(n_point, distances, penalty_scale=10.0):
    x = Array.create('c', shape=(n_point, n_point), vartype='BINARY')

    city_once = 0
    for i in range(n_point):
        city_once += Constraint((sum(x[i, k] for k in range(n_point)) - 1) ** 2, label=f"city_{i}")

    pos_once = 0
    for k in range(n_point):
        pos_once += Constraint((sum(x[i, k] for i in range(n_point)) - 1) ** 2, label=f"pos_{k}")

    dist_term = 0
    for k in range(n_point):
        kp1 = (k + 1) % n_point
        for i in range(n_point):
            xi = x[i, k]
            for j in range(n_point):
                dij = distances[i][j]
                if dij != 0:
                    dist_term += dij * xi * x[j, kp1]

    A = Placeholder("A")
    H = dist_term + A * (city_once + pos_once)

    model = H.compile()
    max_d = float(np.max(distances)) if n_point > 1 else 1.0
    feed = {"A": penalty_scale * max_d}
    bqm = model.to_bqm(feed_dict=feed)
    return model, bqm, feed

In [ ]:

for n_point in range(1, 11):
    distance = TSP_instance(n_point-1)
    for j in range(1, 11):
        pyqubo_model, pyqubo_bqm, pyqubo_feed_dict = tsp_pyqubo(n_point, distances, penalty_scale=1)
        sampleset = SimulatedAnnealingSampler().sample(pyqubo_bqm, num_reads = 2000) # , num_sweeps = 2000
        samples_df = sampleset.to_pandas_dataframe(sample_column=True)
        samples_df = samples_df.sort_values(by='energy', ascending = True)
        samples_df = samples_df.reset_index(drop=True)
        best_sample, best_energy, is_feasible, route_cost, success_rate = check(n_point=n_point-1, df=samples_df)
        

In [ ]:
import math, random

In [ ]:
class City:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def distance(self, city):
        return math.hypot(self.x - city.x, self.y - city.y)

    def __repr__(self):
        return f"({self.x}, {self.y})"


In [ ]:
# 2. City 객체로 변환
cities = [City(x, y) for x, y in zip(df['x'], df['y'])]

In [ ]:
class Fitness:
    def __init__(self, route):
        self.route = route
        self.distance = 0
        self.fitness = 0.0

    def path_cost(self):
        if self.distance == 0:
            distance = 0
            for index, city in enumerate(self.route):
                distance += city.distance(self.route[(index + 1) % len(self.route)])
            self.distance = distance
        return self.distance

    def path_fitness(self):
        if self.fitness == 0:
            self.fitness = 1 / float(self.path_cost())
        return self.fitness


class GeneticAlgorithm:
    def __init__(self, iterations, population_size, cities, elites_num, mutation_rate,
                 greedy_seed=0, roulette_selection=True, plot_progress=True):
        self.plot_progress = plot_progress
        self.roulette_selection = roulette_selection
        self.progress = []
        self.mutation_rate = mutation_rate
        self.cities = cities
        self.elites_num = elites_num
        self.iterations = iterations
        self.population_size = population_size
        self.greedy_seed = greedy_seed

        self.population = self.initial_population()
        self.average_path_cost = 1
        self.ranked_population = None

    def best_chromosome(self):
        return self.ranked_population[0][0]

    def best_distance(self):
        return 1 / self.ranked_population[0][1]

    def random_route(self):
        return random.sample(self.cities, len(self.cities))

    def initial_population(self):
        p1 = [self.random_route() for _ in range(self.population_size - self.greedy_seed)]
        greedy_population = [greedy_route(start_index % len(self.cities), self.cities)
                             for start_index in range(self.greedy_seed)]
        return [*p1, *greedy_population]

    def rank_population(self):
        fitness = [(chromosome, Fitness(chromosome).path_fitness()) for chromosome in self.population]
        self.ranked_population = sorted(fitness, key=lambda f: f[1], reverse=True)

    def selection(self):
        selections = [self.ranked_population[i][0] for i in range(self.elites_num)]
        if self.roulette_selection:
            # ✅ 수정된 코드
            df = pd.DataFrame(self.ranked_population, columns=["chromosome", "fitness"])
            self.average_path_cost = sum(1 / df.fitness) / len(df.fitness)
            df['cum_sum'] = df.fitness.cumsum()
            df['cum_perc'] = 100 * df.cum_sum / df.fitness.sum()

            for _ in range(0, self.population_size - self.elites_num):
                pick = 100 * random.random()
                for i in range(0, len(self.ranked_population)):
                    if pick <= df.iat[i, 3]:  # df['cum_perc']
                        selections.append(self.ranked_population[i][0])
                        break
        else:
            for _ in range(0, self.population_size - self.elites_num):
                pick = random.randint(0, self.population_size - 1)
                selections.append(self.ranked_population[pick][0])
        self.population = selections


    @staticmethod
    def produce_child(parent1, parent2):
        gene_1 = random.randint(0, len(parent1))
        gene_2 = random.randint(0, len(parent1))
        gene_1, gene_2 = min(gene_1, gene_2), max(gene_1, gene_2)
        child = [parent1[i] for i in range(gene_1, gene_2)]
        child.extend([gene for gene in parent2 if gene not in child])
        return child

    def generate_population(self):
        length = len(self.population) - self.elites_num
        children = self.population[:self.elites_num]
        for i in range(0, length):
            child = self.produce_child(self.population[i],
                                       self.population[(i + random.randint(1, self.elites_num)) % length])
            children.append(child)
        return children

    def mutate(self, individual):
        for index, city in enumerate(individual):
            if random.random() < max(0, self.mutation_rate):
                sample_size = min(min(max(3, self.population_size // 5), 100), len(individual))
                random_sample = random.sample(range(len(individual)), sample_size)
                sorted_sample = sorted(random_sample,
                                       key=lambda c_i: individual[c_i].distance(individual[index - 1]))
                random_close_index = random.choice(sorted_sample[:max(sample_size // 3, 2)])
                individual[index], individual[random_close_index] = \
                    individual[random_close_index], individual[index]
        return individual

    def next_generation(self):
        self.rank_population()
        self.selection()
        self.population = self.generate_population()
        self.population[self.elites_num:] = [self.mutate(chromosome)
                                             for chromosome in self.population[self.elites_num:]]

    def run(self):
        if self.plot_progress:
            plt.ion()
        for ind in range(0, self.iterations):
            self.next_generation()
            self.progress.append(self.best_distance())
            if self.plot_progress and ind % 10 == 0:
                self.plot()
            elif not self.plot_progress and ind % 10 == 0:
                print(self.best_distance())
        if self.plot_progress:
            self.plot()

    def plot(self):
        print(self.best_distance())
        fig = plt.figure(0)
        plt.plot(self.progress, 'g')
        fig.suptitle('genetic algorithm generations')
        plt.ylabel('Distance')
        plt.xlabel('Generation')

        x_list, y_list = [], []
        for city in self.best_chromosome():
            x_list.append(city.x)
            y_list.append(city.y)
        x_list.append(self.best_chromosome()[0].x)
        y_list.append(self.best_chromosome()[0].y)
        fig = plt.figure(1)
        fig.clear()
        fig.suptitle('genetic algorithm TSP')
        plt.plot(x_list, y_list, 'ro')
        plt.plot(x_list, y_list, 'g')

        if self.plot_progress:
            plt.draw()
            plt.pause(0.05)
        plt.show()


def greedy_route(start_index, cities):
    unvisited = cities[:]
    del unvisited[start_index]
    route = [cities[start_index]]
    while len(unvisited):
        index, nearest_city = min(enumerate(unvisited), key=lambda item: item[1].distance(route[-1]))
        route.append(nearest_city)
        del unvisited[index]
    return route


if __name__ == '__main__':
    #cities = read_cities(64)
    genetic_algorithm = GeneticAlgorithm(cities=cities, iterations=10, population_size=100,
                                         elites_num=20, mutation_rate=0.008, greedy_seed=1,
                                         roulette_selection=True, plot_progress=True)
    genetic_algorithm.run()
    print(genetic_algorithm.best_distance())
    genetic_algorithm.plot()
    plt.show(block=True)

In [29]:
# ============================================================
# TSP: MIQP (no y) + Genetic Algorithm (GA) with 10-run averages for N=1..10
# - TSP_instance: fixed seed(0), depot=(0,0)
# - GA runs: 10 repeats with different seeds, average & std
# - Output: CSV with MIQP & GA summaries side-by-side
# ============================================================
# pip install docplex scipy tqdm numpy pandas matplotlib

import time, math, random
import numpy as np
import pandas as pd
from tqdm import tqdm
from typing import Optional
from scipy.spatial import distance_matrix
import matplotlib.pyplot as plt  # GA plot optional

# ------------------------------------------------------------
# 0) Instance (고정 seed=0, depot=(0,0))
# ------------------------------------------------------------
def TSP_instance(n_customer: int, seed: int = 0):

    n_point = n_customer + 1
    assert n_point >= 1

    np.random.seed(0)  # 고정

    df = pd.DataFrame({
        'x': np.random.randint(0, 100, n_point),
        'y': np.random.randint(0, 100, n_point),
    })
    df.loc[0, ['x','y']] = (0, 0)  # depot

    D = distance_matrix(df[['x','y']].values, df[['x','y']].values)
    return np.asarray(D, dtype=float)

def make_df_same_as_TSP(n_customer: int):
    """GA 좌표용 df (TSP_instance와 동일 로직, seed=0 고정)."""
    n_point = n_customer + 1
    assert n_point >= 1
    np.random.seed(0)
    df = pd.DataFrame({
        'x': np.random.randint(0, 100, n_point),
        'y': np.random.randint(0, 100, n_point),
    })
    df.loc[0, ['x','y']] = (0, 0)
    return df


# ------------------------------------------------------------
# 2) Genetic Algorithm (GA)
# ------------------------------------------------------------
class City:
    def __init__(self, x, y, idx=None):
        self.x = float(x)
        self.y = float(y)
        self.idx = idx  # 원래 인덱스(0=depot 등)

    def distance(self, city):
        return math.hypot(self.x - city.x, self.y - city.y)

    def __repr__(self):
        return f"[{self.idx}]({self.x:.1f},{self.y:.1f})" if self.idx is not None else f"({self.x:.1f},{self.y:.1f})"

class Fitness:
    def __init__(self, route):
        self.route = route
        self._dist = None
        self._fit = None

    def path_cost(self):
        if self._dist is None:
            dist = 0.0
            n = len(self.route)
            for i in range(n):
                dist += self.route[i].distance(self.route[(i + 1) % n])
            self._dist = dist
        return self._dist

    def path_fitness(self):
        if self._fit is None:
            self._fit = 1.0 / (self.path_cost() + 1e-12)
        return self._fit

def greedy_route(start_index, cities):
    unvisited = cities[:]
    start_city = unvisited.pop(start_index)
    route = [start_city]
    while unvisited:
        idx, nearest = min(enumerate(unvisited), key=lambda it: it[1].distance(route[-1]))
        route.append(nearest)
        unvisited.pop(idx)
    return route

class GeneticAlgorithm:
    def __init__(self, iterations, population_size, cities, elites_num, mutation_rate,
                 greedy_seed=0, roulette_selection=True, plot_progress=False, seed=42):
        self.plot_progress = plot_progress
        self.roulette_selection = roulette_selection
        self.progress = []
        self.mutation_rate = float(mutation_rate)
        self.cities = list(cities)
        self.elites_num = int(elites_num)
        self.iterations = int(iterations)
        self.population_size = int(population_size)
        self.greedy_seed = int(greedy_seed)

        random.seed(seed)
        np.random.seed(seed)

        self.population = self.initial_population()
        self.average_path_cost = 0.0
        self.ranked_population = None

    def best_chromosome(self):
        return self.ranked_population[0][0]

    def best_distance(self):
        return 1.0 / self.ranked_population[0][1]

    def random_route(self):
        return random.sample(self.cities, len(self.cities))

    def initial_population(self):
        p1 = [self.random_route() for _ in range(self.population_size - self.greedy_seed)]
        greedy_population = [greedy_route(start_index % len(self.cities), self.cities)
                             for start_index in range(self.greedy_seed)]
        return [*p1, *greedy_population]

    def rank_population(self):
        fitness = [(chrom, Fitness(chrom).path_fitness()) for chrom in self.population]
        self.ranked_population = sorted(fitness, key=lambda f: f[1], reverse=True)

    def selection(self):
        selections = [self.ranked_population[i][0] for i in range(self.elites_num)]
        if self.roulette_selection:
            df = pd.DataFrame(self.ranked_population, columns=["chromosome", "fitness"])
            self.average_path_cost = float((1.0 / df.fitness).mean())
            df['cum_sum'] = df.fitness.cumsum()
            df['cum_perc'] = 100.0 * df.cum_sum / df.fitness.sum()
            for _ in range(self.population_size - self.elites_num):
                pick = 100.0 * random.random()
                i = int((df['cum_perc'] >= pick).idxmax())
                selections.append(self.ranked_population[i][0])
        else:
            for _ in range(self.population_size - self.elites_num):
                pick = random.randint(0, len(self.ranked_population) - 1)
                selections.append(self.ranked_population[pick][0])
        self.population = selections

    @staticmethod
    def produce_child(parent1, parent2):
        n = len(parent1)
        a, b = random.randint(0, n), random.randint(0, n)
        a, b = min(a, b), max(a, b)
        child = [parent1[i] for i in range(a, b)]
        child.extend([g for g in parent2 if g not in child])
        return child

    def generate_population(self):
        length = len(self.population) - self.elites_num
        children = self.population[:self.elites_num]
        for i in range(length):
            p1 = self.population[i]
            p2 = self.population[(i + random.randint(1, max(1, self.elites_num))) % len(self.population)]
            child = self.produce_child(p1, p2)
            children.append(child)
        return children

    def mutate(self, individual):
        n = len(individual)
        for idx in range(n):
            if random.random() < max(0.0, self.mutation_rate):
                sample_size = min(min(max(3, self.population_size // 5), 100), n)
                chosen = random.sample(range(n), sample_size)
                chosen_sorted = sorted(chosen, key=lambda ci: individual[ci].distance(individual[idx - 1]))
                near_idx = random.choice(chosen_sorted[:max(sample_size // 3, 2)])
                individual[idx], individual[near_idx] = individual[near_idx], individual[idx]
        return individual

    def next_generation(self):
        self.rank_population()
        self.selection()
        self.population = self.generate_population()
        self.population[self.elites_num:] = [self.mutate(ch) for ch in self.population[self.elites_num:]]

    def run(self):
        for it in range(self.iterations):
            self.next_generation()
            self.progress.append(self.best_distance())

# ---- GA 반복 실행 (10회 평균) ----
def run_repeats_ga(cities, runs: int = 10,
                   iterations=500, population_size=150, elites_num=20, mutation_rate=0.02,
                   greedy_seed=1, roulette_selection=True, plot_progress=False):
    dists, times = [], []
    for j in tqdm(range(runs), desc="GA runs", leave=False):
        ga = GeneticAlgorithm(
            iterations=iterations,
            population_size=population_size,
            cities=cities,
            elites_num=elites_num,
            mutation_rate=mutation_rate,
            greedy_seed=greedy_seed,
            roulette_selection=roulette_selection,
            plot_progress=plot_progress,
            seed=42 + j  # 매 반복 다른 난수 시드
        )
        t0 = time.time()
        ga.run()
        elapsed_ms = (time.time() - t0) * 1000.0
        best = ga.best_chromosome()
        cost = Fitness(best).path_cost()
        dists.append(cost)
        times.append(elapsed_ms)

    df = pd.DataFrame(dict(distance=dists, time_ms=times))
    summary = dict(
        ga_dist_mean=float(df["distance"].mean()) if not df.empty else np.nan,
        ga_dist_std=float(df["distance"].std(ddof=1)) if len(df) > 1 else np.nan,
        ga_time_mean_ms=float(df["time_ms"].mean()) if not df.empty else np.nan,
        ga_time_std_ms=float(df["time_ms"].std(ddof=1)) if len(df) > 1 else np.nan,
    )
    return summary, df

# ------------------------------------------------------------
# 3) Main: N=1..10에서 MIQP & GA 각각 10회 평균 요약
# ------------------------------------------------------------
if __name__ == "__main__":
    rows = []
    runs = 10
    # GA 하이퍼파라미터(원하면 조정)
    ga_kwargs = dict(iterations=500, population_size=150, elites_num=20,
                     mutation_rate=0.02, greedy_seed=1, roulette_selection=True, plot_progress=False)

    for N in tqdm(range(1, 11), desc="N loop (MIQP & GA summaries)"):
        # MIQP용 거리행렬
        D = TSP_instance(N, seed=0)
        #miqp_summary, _, _ = run_repeats_miqp(D, runs=runs)

        # GA용 좌표 & City
        df = make_df_same_as_TSP(N)
        cities = [City(x, y, idx=i) for i, (x, y) in enumerate(zip(df['x'], df['y']))]
        ga_summary, _ = run_repeats_ga(cities, runs=runs, **ga_kwargs)

        row = dict(N=N, n_point=N+1)
        #row.update(miqp_summary)
        row.update(ga_summary)
        rows.append(row)

    summary_df = pd.DataFrame(rows).sort_values("N").reset_index(drop=True)
    #print("\n--- MIQP vs GA summary (10 runs each, N=1..10) ---")
    print(summary_df)

    # 저장
    summary_df.to_csv("tsp_miqp_vs_ga_summary_N1to10_10runs.csv", index=False)


N loop (MIQP & GA summaries): 100%|████████████████████████████████████████████████████| 10/10 [04:23<00:00, 26.33s/it]

    N  n_point  ga_dist_mean   ga_dist_std  ga_time_mean_ms  ga_time_std_ms
0   1        2    163.682620  2.995911e-14      2408.358574       12.386305
1   2        3    206.911077  0.000000e+00      2480.262947       23.557190
2   3        4    238.059641  2.995911e-14      2539.412284       15.832176
3   4        5    249.433830  5.013118e-14      2568.696833       17.964647
4   5        6    271.749465  0.000000e+00      2620.024276       22.739784
5   6        7    327.266489  5.991822e-14      2665.155077       11.844077
6   7        8    265.244459  5.359249e-14      2696.379328        7.640320
7   8        9    305.843631  2.679624e-14      2742.690134       15.512196
8   9       10    342.852479  5.991822e-14      2773.541164       32.037405
9  10       11    297.391534  3.281856e-14      2812.304378        9.580405
